In [91]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [92]:
df = pd.read_csv("words2.csv")

In [93]:
def clean_text(input_string):
    def replace_punctuation_with_space(input_string):
        return re.sub(r'[\.\!\-\_]', ' ', input_string)

    def split_string(input_string, chunk_size):
        return [input_string[i:i+chunk_size] for i in range(0, len(input_string), chunk_size)]

    no_symbols = replace_punctuation_with_space(input_string).lower()
    no_symbols = no_symbols.replace(" ", "")
    # tokenized = re.split("\W+", no_symbols.lower())
    tokenized = split_string(no_symbols, 3)
    return tokenized

In [94]:
# count_vect = CountVectorizer(ngram_range=(1,1), analyzer=clean_text)
count_vect = TfidfVectorizer(analyzer=clean_text)

vectorizer = count_vect.fit(df['text'])
X = vectorizer.transform(df['text'])

tokenized_df = pd.DataFrame(X.toarray(), columns=count_vect.get_feature_names())
tokenized_df.head()
# tokenized_df.to_csv("tokenized.csv")

/home/max/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,1,2,],aco,act,acy,add,ade,ado,adv,...,y,yco,yho,yid,yis,yna,ys,z,zip,zon
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
y_labels = df['label'].drop_duplicates().to_list()

l_encoder = LabelEncoder()
l_encoder.fit(y_labels)

y = l_encoder.transform(df['label'])
print(l_encoder.classes_)

y = to_categorical(y)
y.shape

['address' 'city' 'country' 'email' 'housenumber' 'lat' 'location' 'lon'
 'opening_hours' 'phone' 'placename' 'postcode' 'ref' 'state' 'store_url'
 'street' 'unknown']


(285, 17)

In [96]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import gradient_descent_v2
from keras.losses import categorical_crossentropy
from keras.metrics import Accuracy, Precision, accuracy
from sklearn.model_selection import train_test_split

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [98]:
val, input_n = X_train.shape
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(213, 394)
(72, 394)
(213, 17)
(72, 17)


In [99]:
hidden_n = 50
model = Sequential()
model.add(Dense(hidden_n, activation="relu", input_shape=(input_n,)))
model.add(Dense(17, activation="sigmoid", input_shape=(hidden_n,)))

optimizer = gradient_descent_v2.SGD(learning_rate=0.3)
model.compile(
    optimizer=optimizer,
    metrics=[
        Accuracy(),
        Precision(),
    ],
    loss=categorical_crossentropy
)
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 50)                19750     
                                                                 
 dense_18 (Dense)            (None, 17)                867       
                                                                 
Total params: 20,617
Trainable params: 20,617
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.fit(X_train.toarray(), y_train, epochs=100)

Epoch 1/100


7/7 [==============================] - 0s 969us/step - loss: 2.7927 - accuracy: 0.0000e+00 - precision_9: 0.0778
Epoch 2/100
7/7 [==============================] - 0s 906us/step - loss: 2.6726 - accuracy: 0.0000e+00 - precision_9: 0.1116
Epoch 3/100
7/7 [==============================] - 0s 911us/step - loss: 2.5737 - accuracy: 0.0000e+00 - precision_9: 0.1127
Epoch 4/100
7/7 [==============================] - 0s 882us/step - loss: 2.5048 - accuracy: 0.0000e+00 - precision_9: 0.1120
Epoch 5/100
7/7 [==============================] - 0s 802us/step - loss: 2.4523 - accuracy: 0.0000e+00 - precision_9: 0.1205
Epoch 6/100
7/7 [==============================] - 0s 860us/step - loss: 2.4110 - accuracy: 0.0000e+00 - precision_9: 0.1228
Epoch 7/100
7/7 [==============================] - 0s 801us/step - loss: 2.3703 - accuracy: 0.0000e+00 - precision_9: 0.1159
Epoch 8/100
7/7 [==============================] - 0s 884us/step - loss: 2.3320 - accuracy: 0.0000e+00 - precision_9: 0.1149
Epoch 9/100


In [101]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predictions = model.predict(X_test.toarray())

y_pred = np.array([np.argmax(pred) for pred in predictions])
y_test_a = np.array([np.argmax(pred) for pred in y_test])



# assert y_test_a.shape == y_pred.shape
print(accuracy_score(y_test_a, y_pred))
print(precision_score(y_test_a, y_pred, average="weighted"))
print(f1_score(y_test_a, y_pred, average="weighted"))

3/3 [==============================] - 0s 2ms/step
0.6527777777777778
0.689263963161022
0.6599375061331583


/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [102]:
word_vect = vectorizer.transform(["region"]).toarray()

prediction = model.predict(word_vect)

print(l_encoder.classes_[np.argmax(prediction[0])])
list(zip(l_encoder.classes_, prediction[0]))

1/1 [==============================] - 0s 24ms/step
state


[('address', 0.3506005),
 ('city', 0.6660605),
 ('country', 0.2535316),
 ('email', 0.45115393),
 ('housenumber', 0.6001632),
 ('lat', 0.2998368),
 ('location', 0.5115683),
 ('lon', 0.26927492),
 ('opening_hours', 0.6887708),
 ('phone', 0.25778553),
 ('placename', 0.33739865),
 ('postcode', 0.34698915),
 ('ref', 0.09894715),
 ('state', 0.9898834),
 ('store_url', 0.41442743),
 ('street', 0.23485182),
 ('unknown', 0.84018636)]